In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/auto-sales-data/Auto Sales data.csv


In [2]:
# Importing necessary libraries
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [3]:
# Creating an XGBoost Regressor Model
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
xgb_model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [4]:
# Loading the dataset
df = pd.read_csv('/kaggle/input/auto-sales-data/Auto Sales data.csv')

In [5]:
df = df.drop(['ORDERDATE','POSTALCODE','CONTACTLASTNAME', 'CONTACTFIRSTNAME','ORDERNUMBER','PHONE', 'ADDRESSLINE1','POSTALCODE'],axis=1)

#df['ORDERDATE'] = pd.to_datetime(df['ORDERDATE'],yearfirst = True ) # changing the date time format to a suitable format

df.head()

,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,DAYS_SINCE_LASTORDER,STATUS,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,CITY,COUNTRY,DEALSIZE
0,30,95.70,2,2871.00,828,Shipped,Motorcycles,95,S10_1678,Land of Toys Inc.,NYC,USA,Small
1,34,81.35,5,2765.90,757,Shipped,Motorcycles,95,S10_1678,Reims Collectables,Reims,France,Small
2,41,94.74,2,3884.34,703,Shipped,Motorcycles,95,S10_1678,Lyon Souveniers,Paris,France,Medium
3,45,83.26,6,3746.70,649,Shipped,Motorcycles,95,S10_1678,Toys4GrownUps.com,Pasadena,USA,Medium
4,36,96.66,1,3479.76,586,Shipped,Motorcycles,95,S10_1678,Technics Stores Inc.,Burlingame,USA,Medium


In [6]:
df.nunique()

QUANTITYORDERED           58
PRICEEACH               1843
ORDERLINENUMBER           18
SALES                   2690
DAYS_SINCE_LASTORDER    1832
STATUS                     6
PRODUCTLINE                7
MSRP                      80
PRODUCTCODE              109
CUSTOMERNAME              89
CITY                      71
COUNTRY                   19
DEALSIZE                   3
dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2747 entries, 0 to 2746
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   QUANTITYORDERED       2747 non-null   int64  
 1   PRICEEACH             2747 non-null   float64
 2   ORDERLINENUMBER       2747 non-null   int64  
 3   SALES                 2747 non-null   float64
 4   DAYS_SINCE_LASTORDER  2747 non-null   int64  
 5   STATUS                2747 non-null   object 
 6   PRODUCTLINE           2747 non-null   object 
 7   MSRP                  2747 non-null   int64  
 8   PRODUCTCODE           2747 non-null   object 
 9   CUSTOMERNAME          2747 non-null   object 
 10  CITY                  2747 non-null   object 
 11  COUNTRY               2747 non-null   object 
 12  DEALSIZE              2747 non-null   object 
dtypes: float64(2), int64(4), object(7)
memory usage: 279.1+ KB


In [8]:
label_encoder = LabelEncoder()

df['STATUS'] = label_encoder.fit_transform(df['STATUS'])
df['PRODUCTLINE'] = label_encoder.fit_transform(df['PRODUCTLINE'])
df['PRODUCTCODE'] = label_encoder.fit_transform(df['PRODUCTCODE'])
df['CUSTOMERNAME'] = label_encoder.fit_transform(df['CUSTOMERNAME'])
df['CITY'] = label_encoder.fit_transform(df['CITY'])
df['COUNTRY'] = label_encoder.fit_transform(df['COUNTRY'])
df['DEALSIZE'] = label_encoder.fit_transform(df['DEALSIZE'])

df.head()

,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,DAYS_SINCE_LASTORDER,STATUS,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,CITY,COUNTRY,DEALSIZE
0,30,95.70,2,2871.00,828,5,1,95,0,44,41,18,2
1,34,81.35,5,2765.90,757,5,1,95,0,63,55,6,2
2,41,94.74,2,3884.34,703,5,1,95,0,45,51,6,1
3,45,83.26,6,3746.70,649,5,1,95,0,82,52,18,1
4,36,96.66,1,3479.76,586,5,1,95,0,76,12,18,1


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2747 entries, 0 to 2746
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   QUANTITYORDERED       2747 non-null   int64  
 1   PRICEEACH             2747 non-null   float64
 2   ORDERLINENUMBER       2747 non-null   int64  
 3   SALES                 2747 non-null   float64
 4   DAYS_SINCE_LASTORDER  2747 non-null   int64  
 5   STATUS                2747 non-null   int64  
 6   PRODUCTLINE           2747 non-null   int64  
 7   MSRP                  2747 non-null   int64  
 8   PRODUCTCODE           2747 non-null   int64  
 9   CUSTOMERNAME          2747 non-null   int64  
 10  CITY                  2747 non-null   int64  
 11  COUNTRY               2747 non-null   int64  
 12  DEALSIZE              2747 non-null   int64  
dtypes: float64(2), int64(11)
memory usage: 279.1 KB


In [10]:
# 'SALES' is the target variable
X = df.drop('SALES', axis=1)  # Features
y = df['SALES']  # Target variable

In [11]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
# Training the model
xgb_model.fit(X_train, y_train)# enable_categorical = True)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [13]:
# Making predictions on the test set
y_pred = xgb_model.predict(X_test)
y_pred
#len(y_pred)

array([ 2403.1553 ,  2072.4946 ,  2555.7063 ,  2502.582  ,  1669.9224 ,
        3953.5376 ,  2203.6199 ,  7761.205  ,  3191.2153 ,  5964.5576 ,
        3091.8733 ,  2302.9805 ,  1310.5719 ,  3247.6428 ,  3205.15   ,
        5584.158  ,  6689.223  ,  2867.3447 ,  6834.1245 ,  2304.5242 ,
        6847.637  ,  2519.09   ,  2433.2046 ,  2089.601  ,  1549.6914 ,
        1229.718  ,  3109.7095 ,  6419.448  ,  2805.951  ,  4854.442  ,
        3229.3657 ,  5867.267  ,  5511.0933 ,  4163.9717 ,  4565.2837 ,
        3515.6213 ,  5144.1333 ,  3991.424  ,  2240.0688 ,  1854.4648 ,
        1811.3522 ,  1170.1559 ,  2299.975  ,  4466.585  ,  2803.166  ,
        2306.382  ,  3824.1335 ,  1957.6715 ,  7504.7627 ,  3117.569  ,
        3280.136  ,  1871.8859 ,  2144.5132 ,  1332.3097 ,  5980.693  ,
        2379.156  ,  6688.3726 ,  6303.256  ,  1621.0576 ,  1630.3422 ,
        3913.6206 ,  6602.1636 ,  3497.5007 ,  4207.74   ,  6065.947  ,
        7488.361  ,  1536.791  ,  2809.0823 ,  3918.8967 ,  2896

In [14]:
y_test

2678    2466.70
2713    2060.37
2261    2514.60
2519    2451.84
1561    1729.65
         ...   
1398    1955.88
1777    2081.76
2175    1329.40
179     8594.52
1018    3353.02
Name: SALES, Length: 550, dtype: float64

In [15]:
# Evaluating the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

Mean Absolute Error: 59.223857479580964
Mean Squared Error: 11492.09157470519
R-squared: 0.996230050319726


**Hyperparameter Tuning**

In [16]:
from sklearn.model_selection import GridSearchCV

In [17]:
# Define the hyperparameters grid to search
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
}

In [18]:
# Use GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='neg_mean_absolute_error', cv=3, verbose=2, n_jobs=-1)

In [19]:
# Fit the grid search to the data
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 243 candidates, totalling 729 fits
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8; total time=   0.1s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=1.0; total time=   0.1s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=200, subsample=0.8; total time=   0.2s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=200, subsample=0.9; total time=   0.2s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=300, subsample=0.8; total time=   0.3s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=300, subsample=0.9; total time=   0.3s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=300, subsample=1.0; total time=   0.3s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=4, n_estimators=100, subsample=0.9; total time=   0.2s
[CV] END 

GridSearchCV(cv=3,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=0.1, ma...
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=100,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=42, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.8, 0.9, 1.0],
                         'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 4, 5],
                         'n_estimators': [100, 200, 300],
                         'subsample': [0.8, 0.9, 1.0]},
             scoring='neg_mean_absolute_error', verbose=2)

In [20]:
# Get the best parameters
best_params = grid_search.best_params_

best_params

{'colsample_bytree': 1.0,
 'learning_rate': 0.1,
 'max_depth': 5,
 'n_estimators': 300,
 'subsample': 1.0}

In [21]:
# Create the final model with the best parameters
final_model = XGBRegressor(**best_params)

final_model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=1.0, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=300, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [22]:
# Train the final model
final_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=1.0, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=300, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [23]:
# Make predictions on the test set
y_pred = final_model.predict(X_test)

y_pred

array([ 2444.2522 ,  2066.012  ,  2520.02   ,  2551.2366 ,  1699.5748 ,
        4022.0957 ,  2175.8145 ,  7679.6587 ,  3182.4526 ,  5950.713  ,
        3127.1987 ,  2338.734  ,  1252.5082 ,  3278.6682 ,  3251.847  ,
        5539.4727 ,  6673.2544 ,  2911.2454 ,  6857.451  ,  2327.3516 ,
        6795.3394 ,  2550.0498 ,  2426.8953 ,  1884.8942 ,  1622.764  ,
        1263.1381 ,  3117.3289 ,  6401.169  ,  2823.5027 ,  4725.5044 ,
        3231.9795 ,  5723.282  ,  5555.397  ,  4226.447  ,  4632.392  ,
        3562.352  ,  5180.158  ,  3912.393  ,  2222.8215 ,  1868.5526 ,
        1774.2852 ,  1181.3376 ,  2332.3506 ,  4407.108  ,  2810.532  ,
        2328.5542 ,  3838.7024 ,  1947.6445 ,  7385.997  ,  3122.34   ,
        3299.4692 ,  1842.1705 ,  2109.4915 ,  1345.3925 ,  5962.681  ,
        2398.177  ,  6726.639  ,  6297.942  ,  1626.659  ,  1706.4624 ,
        3910.965  ,  6674.12   ,  3443.8328 ,  4147.343  ,  6037.333  ,
        7503.062  ,  1534.3463 ,  2802.6038 ,  3929.836  ,  2891

In [24]:
# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Best Hyperparameters: {best_params}')
print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

Best Hyperparameters: {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300, 'subsample': 1.0}
Mean Absolute Error: 54.20415514914773
Mean Squared Error: 10535.270870685503
R-squared: 0.996543932773912


**Before Hyperparameter Tuning**

Mean Absolute Error: 59.223857479580964

Mean Squared Error: 11492.09157470519

R-squared: 0.996230050319726


**After Hyperparameter Tuning**

Mean Absolute Error: 54.20415514914773

Mean Squared Error: 10535.270870685503

R-squared: 0.996543932773912